# Script de migracion de datos

Este programa está diseñado para procesar datos de una hoja de cálculo de Google Sheets, descargándolos, transformándolos y guardándolos en un archivo Excel. La clase principal, GoogleSheetProcessor, maneja todo el flujo de trabajo, desde la obtención de datos hasta su procesamiento y almacenamiento. A continuación se describe el funcionamiento detallado del programa.

## Funcionalidades Principales
1. Inicialización (__init__)
La clase se inicializa con la URL de una hoja de cálculo de Google Sheets. Durante la inicialización, se extraen los identificadores del archivo y de la hoja específica, y se construye la URL para exportar la hoja en formato CSV.

### Atributos:

sheet_url: URL de la hoja de cálculo de Google Sheets.
spreadsheet_id: ID único de la hoja de cálculo.
sheet_id: ID único de la hoja dentro de la hoja de cálculo.
csv_export_url: URL para exportar la hoja en formato CSV.

### Diccionarios predefinidos:

valores: Diccionario que asocia códigos con valores numéricos.
regimen: Diccionario que asocia códigos con sus unidades de medida correspondientes.

2. Extracción de Identificadores
extract_spreadsheet_id(url): Extrae el ID de la hoja de cálculo desde la URL.
extract_sheet_id(url): Extrae el ID de la hoja específica desde la URL.
3. Construcción de la URL de Exportación
construct_csv_export_url(): Construye la URL que permite descargar la hoja de cálculo en formato CSV.
4. Descarga de la Hoja en CSV
download_csv(output_filename='temp_sheet.csv'): Descarga la hoja de cálculo en formato CSV y la guarda con un nombre de archivo especificado (por defecto, temp_sheet.csv).
5. Procesamiento de Datos
process_data(filename="temp_sheet.csv", valores="", regimen=""):
Carga el archivo CSV y realiza una serie de transformaciones y filtrados en los datos, como la conversión de valores, la asignación de unidades, y la reestructuración del DataFrame.
Comprueba que las claves de los diccionarios valores y regimen coinciden, lanzando un AssertionError si no es así.
Filtra los datos para excluir planes y mantener solo las columnas relevantes.
Realiza transformaciones en el DataFrame para preparar la información que se almacenará en Excel.
Retorna los DataFrames df_plan, df_action, df_speciality y filtered_data.
6. Guardar Resultados en Excel
save_to_excel(output_path="Salida.xlsx", filename="mix_plan.csv", valores="", regimen=""):
Llama al método process_data para obtener los DataFrames procesados.
Guarda los DataFrames resultantes en un archivo Excel con hojas separadas para acciones, planes, especialidades y actividades filtradas.
7. Funciones Auxiliares
get_unique(df: pd.DataFrame, column: str): Genera un DataFrame con valores únicos de una columna específica, ajustando los índices.
buscarIndice(df: pd.DataFrame, valor, columna='value'): Busca el índice de un valor específico en un DataFrame y lo retorna como un entero.
Uso del Programa
Inicializar la Clase:

```
processor = GoogleSheetProcessor(sheet_url="https://docs.google.com/spreadsheets/d/.../edit#gid=...")
```

Descargar y Leer CSV:

```
processor.download_csv("datos.csv")
processor.read_csv("datos.csv")
```

Procesar Datos:
```
df_plan, df_action, df_speciality, filtered_data = processor.process_data(filename="datos.csv")
```

Guardar los Resultados en Excel:

```
processor.save_to_excel(output_path="Resultados.xlsx")
```

Este programa es útil para automatizar la descarga y procesamiento de datos desde Google Sheets, permitiendo la preparación de datos en un formato listo para análisis o almacenamiento posterior.

In [6]:
import pandas as pd
import requests

class GoogleSheetProcessor:
    def __init__(self, sheet_url:str):
        self.sheet_url = sheet_url
        self.spreadsheet_id = self.extract_spreadsheet_id(sheet_url)
        self.sheet_id = self.extract_sheet_id(sheet_url)
        self.csv_export_url = self.construct_csv_export_url()

        # Diccionarios originales
        self.valores = {
            "D": 1, "S": 1, "M": 5, "MC": 1, "2M": 2, "T": 3, "4M": 4, "SE": 6,
            "8M": 8, "A": 1, "1.5A": 18, "2A": 2, "3A": 3, "4A": 4, "5A": 5,
            "6A": 6, "8A": 8, "10A": 10, "1000": 1000, "6000": 6000, "22500": 22500,
            "40000": 40000, "55000": 55000
        }

        self.regimen = {
            "D": 'dia', "S": 'semana', "M": 'semana', "MC": 'mes', "2M": 'mes', "T": 'mes',
            "4M": 'mes', "SE": 'mes', "8M": 'mes', "A": 'Año', "1.5A": 'mes', "2A": 'Año',
            "3A": 'Año', "4A": 'Año', "5A": 'Año', "6A": 'Año', "8A": 'Año', "10A": 'Año',
            "1000": 'horas', "6000": 'horas', "22500": 'horas', "40000": 'horas', "55000": 'horas'
        }

    def extract_spreadsheet_id(self, url):
        return url.split('/d/')[1].split('/')[0]

    def extract_sheet_id(self, url):
        return url.split('gid=')[1]

    def construct_csv_export_url(self):
        return f"https://docs.google.com/spreadsheets/d/{self.spreadsheet_id}/export?format=csv&gid={self.sheet_id}"

    def download_csv(self, output_filename='temp_sheet.csv'):
        # Descarga el archivo CSV y lo guarda temporalmente
        response = requests.get(self.csv_export_url)
        response.raise_for_status()  # Asegurarse de que la solicitud fue exitosa
        with open(output_filename, 'wb') as f:
            f.write(response.content)
        return output_filename
    def get_unique(self, df: pd.DataFrame, column: str):
        """
        Obtiene un DataFrame con valores únicos de la columna 'Column', con índices ajustados.

        Returns:
        pd.DataFrame: Un DataFrame con valores únicos de la columna 'Column' y un índice ajustado.
        """
        df[column] = df[column].str.strip()
        df = df[df[column].notnull()]
        df_unique = pd.DataFrame(df[column].unique(), columns=['value'])
        df_unique.index = df_unique.index + 1
        return df_unique
    def buscarIndice(self, df:pd.DataFrame, valor,columna='value'):
        return int (df[df[columna]==valor].index[0])


    def read_csv(self, filename="temp_sheet.csv"):
        # Lee el archivo CSV usando pandas
        self.df = pd.read_csv(filename)
        self.df.columns = self.df.loc[2, :].to_list()  # la fila 2 como fila
        self.df = self.df.loc[4:, :]   # Obtener desde la fila 4 en adelante
        return self.df

    def process_data(self,filename = "temp_sheet.csv",valores="", regimen=""):        
        df = pd.DataFrame()
        df = pd.read_csv(filename)

        if valores == "":valores = self.valores
        
        if regimen == "":regimen = self.regimen
        
        if valores.keys() != regimen.keys():
            raise AssertionError(f"Las claves no coinciden: {valores.keys()} != {regimen.keys()}")

        #self.df = df.copy(deep=True)

        # Realiza el procesamiento necesario
        # Este es un lugar para incluir toda la lógica de procesamiento
        
        # Suponiendo que el procesamiento produce 'filtered_data' y otros DataFrames
        df_plan = pd.DataFrame()  # Placeholder
        df_action = pd.DataFrame()  # Placeholder
        df_speciality = pd.DataFrame()  # Placeholder
        filtered_data = pd.DataFrame()  # Placeholder        
        print(valores)
        ## convertir a booleano
        df[list(valores.keys())] = df[valores.keys()].applymap(lambda x: True if x == 'TRUE' else False)
        # Obtener la unidades
        parametros = self.regimen
        df['unidad'] = df.apply(lambda row: next((parametros[key] for key in parametros.keys() if key in row and row[key] == True), None), axis=1)
        # Obtener los valores
        parametros = self.valores
        df['valor'] = df.apply(lambda row: next((parametros[key] for key in parametros.keys() if key in row and row[key] == True), None), axis=1)
        # Filtrar las columnas necesarias solamente

        # Quitar planes
        df = df[df['Tipo']!= 'Plan']

        # Mantener solo las columnas necesarias
        columns = ['Plan','Accion','Trabajo','Actividad','Tipo','Parada','Relevancia','Especialidad','valor','unidad']
        df = df[columns]
        # Crear la nueva columna fk_activity que tendra relaciones con las actividades padre
        df['fk_activity']= None
        df['fkc_regime']= None

        # renombrar los nombres de las columnas
        nuevos_nombres = {
            'Plan': 'fk_plan',
            'Accion': 'fk_action',
            'Actividad': 'name',
            'Tipo': 'fkc_activity_type',
            'Relevancia': 'fkc_priority',
            'Especialidad': 'fk_specialty',
            'valor': 'time_interval_value',
            'unidad': 'fk_periodicity_unit',
            'Parada': 'stoppage',
        }
        df.rename(columns=nuevos_nombres, inplace=True)
                # Mantener las columnas del excel en el orden indicado
        columnas_excel = ['fk_activity','fk_plan','fk_action','name','fkc_activity_type','fkc_priority','fk_specialty','fkc_regime','stoppage','time_interval_value','fk_periodicity_unit'] 

        df = df[columnas_excel]
        df_plan = self.get_unique(df,"fk_plan")
        df_action = self.get_unique(df,"fk_action")
        df_speciality = self.get_unique(df,"fk_specialty")
        #df_activity_type = self.get_unique(df,"fkc_activity_type")
        #df_regime = self.get_unique(df,"fkc_regime")

        # Filter the data
        #df = df_raw.copy(deep=True)
        filtered_data = df[(df['fkc_activity_type'] == 'Actividad') | (df['fkc_activity_type'] == 'Tarea')]

        # Add fk_activity column
        filtered_data['fk_activity'] = None

        # Set fk_activity for Tareas based on their parent Actividad
        parent_index = None
        for i, row in filtered_data.iterrows():
            if row['fkc_activity_type'] == 'Actividad':
                parent_index = i
            elif row['fkc_activity_type'] == 'Tarea':
                filtered_data.at[i, 'fk_activity'] = parent_index
        #filtered_data

        filtered_data['fk_plan']= filtered_data['fk_plan'].apply(lambda x: self.buscarIndice(df_plan,x)) 


        return df_plan, df_action, df_speciality, filtered_data


    def save_to_excel(self, output_path="Salida.xlsx",filename="mix_plan.csv",valores="",regimen=""):        
        df_plan, df_action, df_speciality, filtered_data = self.process_data(filename=filename,valores=valores,regimen=regimen)  
        with pd.ExcelWriter(output_path) as writer:
            df_action.to_excel(writer, sheet_name='Acciones')
            df_plan.to_excel(writer, sheet_name='Plan')
            df_speciality.to_excel(writer, sheet_name='Especialidad')
            filtered_data.to_excel(writer, sheet_name='Actividades')


In [7]:
# Combinacion de dataframes



gs1 = GoogleSheetProcessor("https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1199302294")
archivo = "pad.csv"
#gs.download_csv(archivo)
df1 = gs1.read_csv(archivo)

gs2 = GoogleSheetProcessor("https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1199302294")
archivo = "pbd.csv"
#gs.download_csv(archivo)
df2 = gs2.read_csv(archivo)
#df1,df2

# df_plan, df_action, df_speciality, filtered_data = gs.process_data()
#gs.save_to_excel("Salida.xlsx")

In [8]:
# Realizar el merge de ambos planes en un solo dataframe
df_merged = pd.concat([df1,df2],ignore_index=True).reset_index(drop=True)
df_merged.to_csv("mix_plan.csv")


In [9]:
#gs1.process_data(filename="mix_plan.csv",valores=valores,regimen=regimen)
# Diccionarios originales
valores = {
    "D": 1,
    "S": 1,
    "2S": 2,
    "M": 5,
    "MC": 1,
    "2M": 2,
    "T": 3,
    "SE": 6,
    "8M": 8,
    "A": 1,
    "1.5A": 18,
    "2A": 2,
    "3A": 3,
    "4A": 4,
    "5A": 5,
    "6A": 6,
    "8A": 8,
    "10A": 10,
    "1000": 1000,
    "1300": 1300,
    "1800": 1800,
    "6000": 6000,
    "22500": 6000,
    "40000": 40000,
    "55000": 55000,
    "55000C": 55000
}

regimen = {
    "D": 'dia',
    "S": 'semana',
    "2S": 'semana',
    "M": 'semana',
    "MC": 'mes',
    "2M": 'mes',
    "T": 'mes',
    "SE": 'mes',
    "8M": 'mes',
    "A": 'Año',
    "1.5A": 'mes',
    "2A": 'Año',
    "3A": 'Año',
    "4A": 'Año',
    "5A": 'Año',
    "6A": 'Año',
    "8A": 'Año',
    "10A": 'Año',
    "1000": 'horas',
    "1300": 'horas',
    "1800": 'horas',
    "6000": 'horas',
    "22500": 'horas',
    "40000": 'horas',
    "55000": 'horas',
    "55000C": 'ciclos'
}
#df_merged.tail(5)

In [10]:
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
import warnings

import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display

warnings.simplefilter(action='ignore', category=FutureWarning)

lad = widgets.Textarea(value='https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1199302294',placeholder='Plan Maestro LAD',description='Lineas Alta Demanda:',disabled=False,layout=Layout(width='70%',height="200px"))
lbd = widgets.Textarea(value='https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1199302294',placeholder='Plan Maestro LAD',description='Lineas Baja Demanda:',disabled=False,layout=Layout(width='70%',height="200px"))
host = widgets.Text(value='192.168.100.50',placeholder='Host',description='Host:',disabled=False)
basedatos = widgets.Text(value='simyo2',placeholder='BaseDatos',description='BaseDatos',disabled=False)
usuario = widgets.Text(value='mantto',description='Usuario')
password = widgets.Password(value='Sistemas0',description='Password')
button1 = widgets.Button(description="Generar Archivo Excel",button_style='success',layout=Layout(width='20%'))
button2 = widgets.Button(description="Cargar en Base de datos",button_style='danger',layout=Layout(width='20%'))
output = widgets.Output()
salida = widgets.Text(value="Salida.xlsx",description="Nombre:",disabled=False)
accordion = widgets.Accordion(children=[ salida], titles=(['Archivo Salida']))
accordion1 = widgets.Accordion(children=[ usuario,password,host,basedatos], titles=('Usuario','Password','Host','Base de Datos'))

display(lad,lbd,host,usuario,password,accordion,button1, accordion1,button2,output)

def on_button_clicked(b):    
    #print(f"Se hizo click: {usuario.value} ")
    gs1.save_to_excel("output/Salida.xlsx",valores=valores,regimen=regimen,filename="mix_plan.csv")

    with output:
        print("Se Genera archivo excel Salida.xlsx")

button1.on_click(on_button_clicked)
button2.on_click(lambda _: print("Boton 2 accionado"))
#https://ipywidgets.readthedocs.io/en/7.6.3/examples/Widget%20Styling.html

Textarea(value='https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1…

Textarea(value='https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1…

Text(value='192.168.100.50', description='Host:', placeholder='Host')

Text(value='mantto', description='Usuario')

Password(description='Password')

...

Button(button_style='success', description='Generar Archivo Excel', layout=Layout(width='20%'), style=ButtonSt…

Accordion(children=(Text(value='mantto', description='Usuario'), Password(description='Password'), Text(value=…

Button(button_style='danger', description='Cargar en Base de datos', layout=Layout(width='20%'), style=ButtonS…

Output()

{'D': 1, 'S': 1, '2S': 2, 'M': 5, 'MC': 1, '2M': 2, 'T': 3, 'SE': 6, '8M': 8, 'A': 1, '1.5A': 18, '2A': 2, '3A': 3, '4A': 4, '5A': 5, '6A': 6, '8A': 8, '10A': 10, '1000': 1000, '1300': 1300, '1800': 1800, '6000': 6000, '22500': 6000, '40000': 40000, '55000': 55000, '55000C': 55000}
